In [2]:
import pandas as pd
import re
import numpy as np
import requests
from bs4 import BeautifulSoup

In [3]:
df = pd.read_csv('SEC_lit_releases_2017.csv')

In [4]:
pd.set_option('display.max_colwidth', -1)

In [5]:
df.head()

,Release No.,Date,Respondents
0,LR-24022,"Dec. 26, 2017","Gaughran, et al. See also: Complaint"
1,LR-24021,"Dec. 26, 2017","Anvil Partners, Inc. See also: Contempt Order"
2,LR-24020,"Dec. 21, 2017","Robert H. Shapiro, Woodbridge Group of Companies, LLC, et al. See also: SEC Complaint"
3,LR-24019,"Dec. 21, 2017","Stephen C. Peters, VisionQuest Wealth Management, LLC, VisionQuest Capital, LLC, and VQ Wealth, LLC See also: SEC Complaint"
4,LR-24018,"Dec. 20, 2017","Matrix Capital Markets, LLC, Nicholas M. Mitsakos and Courtlin L. Holt-Nguyen"


In [6]:
# make a new column of lowercase release numbers to generate URL column
df['release_no'] = df['Release No.'].str.replace('-', '')
df['release_no'] = df['release_no'].str.lower()

In [7]:
# add the url column based on the url pattern
df['url'] = "https://www.sec.gov/litigation/litreleases/2017/" + df.release_no + ".htm"

In [8]:
# drop the release_no column
df = df.drop(['release_no'], axis=1)

In [9]:
df

,Release No.,Date,Respondents,url
0,LR-24022,"Dec. 26, 2017","Gaughran, et al. See also: Complaint",https://www.sec.gov/litigation/litreleases/2017/lr24022.htm
1,LR-24021,"Dec. 26, 2017","Anvil Partners, Inc. See also: Contempt Order",https://www.sec.gov/litigation/litreleases/2017/lr24021.htm
2,LR-24020,"Dec. 21, 2017","Robert H. Shapiro, Woodbridge Group of Companies, LLC, et al. See also: SEC Complaint",https://www.sec.gov/litigation/litreleases/2017/lr24020.htm
3,LR-24019,"Dec. 21, 2017","Stephen C. Peters, VisionQuest Wealth Management, LLC, VisionQuest Capital, LLC, and VQ Wealth, LLC See also: SEC Complaint",https://www.sec.gov/litigation/litreleases/2017/lr24019.htm
4,LR-24018,"Dec. 20, 2017","Matrix Capital Markets, LLC, Nicholas M. Mitsakos and Courtlin L. Holt-Nguyen",https://www.sec.gov/litigation/litreleases/2017/lr24018.htm
5,LR-24017,"Dec. 20, 2017","Ronald A. Fossum, Jr. and Alonzo R. Cahoon See also: SEC Complaint",https://www.sec.gov/litigation/litreleases/2017/lr24017.htm
6,LR-24016,"Dec. 18, 2017",Jason A. Wallace,https://www.sec.gov/litigation/litreleases/2017/lr24016.htm
7,LR-24015,"Dec. 14, 2017","Lanny Brown, et al. See also: SEC Complaint",https://www.sec.gov/litigation/litreleases/2017/lr24015.htm
8,LR-24014,"Dec. 14, 2017","Justin D. Meadlin and Hyaline Capital Management, LLC",https://www.sec.gov/litigation/litreleases/2017/lr24014.htm
9,LR-24013,"Dec. 14, 2017","Ray C. Davis and Behavioral Recognition Systems, Inc., n/k/a Giant Gray, Inc. See also: SEC Complaint",https://www.sec.gov/litigation/litreleases/2017/lr24013.htm


In [10]:
# using BeautifulSoup loop through each url and pull all the text behind it

def get_release_info(row):
    
    try:
        request = requests.get(row['url'])
        soup = BeautifulSoup(request.text, 'html.parser')
        
        full_text = soup.findAll(text=True)
        
    except:
        return pd.Series({})

    return pd.Series({
        'full_text': full_text,
    })

In [11]:
df_litreleases_17 = df.apply(get_release_info, axis=1).join(df)

In [12]:
df_litreleases_17.head()

full_text  \
0  [HTML PUBLIC "-//W3C//DTD HTML 4.0 Transitional//EN", 
, 
, 
, Gaughran, et al. (Release No. LR-24022; Dec. 26, 2017), 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, Home,  | , Previous Page, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, U.S. SECURITIES AND EXCHANGE COMMISSION, 
, Litigation Release No. 24022 / December 26, 2017, 
, SEC v. Gaughran, et al., No. 17 Civ. 10026 (S.D.N.Y.), 
, SEC Charges Attorney and Accountant with Aiding and Abetting Adviser's $9 Million Theft from Charity Client, Adviser Settles Related Fraud Claims, 
,  Text goes here , 
, The U.S. Securities and Exchange Commission today charged New York-based investment advisor Train Babcock Advisors LLC ("TBA"), a New Jersey-based attorney, and a New York-based accountant, in connection with a fraudulent scheme led by TBA's former principal, John Rogicki, who stole more than $9 million from his advisory client, a charitable foundation established by an elderly widow to donate her estate to health and education causes., 
, TBA, which is in the process of winding down its operations, has agreed to settle the SEC's charges by paying more than $1.7 million in disgorgement, interest, and penalties for these and other violations.  TBA has also agreed to be censured and to withdraw its registration with the SEC as an investment advisor., 
, Separately, the SEC filed a complaint in federal court in Manhattan against one of the foundation's trustees, attorney Robert Gaughran, and its accountant, Kevin Clune, alleging that they aided and abetted the fraud perpetrated by TBA and Rogicki, the foundation's investment adviser, President, and a trustee., 
, According to the SEC's filings, between 2004 and 2016, Rogicki took advantage of his roles as investment adviser and trustee to the foundation by liquidating securities positions in the foundation's advisory account with TBA and misappropriating a total of more than $9 million from the foundation for his own personal benefit.  The SEC's complaint alleges that Gaughran and Clune aided and abetted TBA's and Rogicki's fraudulent scheme.  While serving as a trustee of the foundation, Gaughran allegedly accepted outsized fees and ignored glaring signs of Rogicki's theft that were apparent from the foundation's brokerage statements and other documents that he regularly reviewed.  Gaughran also drafted the trust and estate papers that put Rogicki in charge of the charitable foundation and knew the size of the estate that should have flowed to the charitable foundation, but for Rogicki's misappropriation of $9 million of its assets.  According to the SEC's complaint, Clune, an accountant who performed tax work for both the estate that created the charitable foundation and the charitable foundation itself, ignored multiple and repeated red flags revealing Rogicki's and TBA's fraudulent scheme.  In addition, both Gaughran and Clune were actively involved in concealing the theft by providing false information to an outside audit firm during a surprise examination of the charitable foundation that was conducted in 2014., 
, On October 19, 2017, the SEC filed a civil injunctive action in federal district court against Rogicki.  In a parallel criminal action, Rogicki pleaded guilty to criminal charges brought against him by the Manhattan District Attorney on October 19, 2017.  On December 14, 2017, Rogicki was sentenced to serve 30 to 90 months in prison and ordered to pay $6,728,391.77 to the foundation, of which he forfeited $2.5 million, in connection with his criminal plea.  The SEC also previously instituted administrative proceedings against former TBA CEO Brian J. Keenan on September 26, 2017, in connection with his misappropriation of assets from a separate client account.  The administrative proceedings against Keenan were settled on October 23, 2017., 
, The SEC's complaint, filed in the United States District Court for the Southern District of New Y

In [13]:
df_litreleases_17.dtypes

full_text      object
Release No.    object
Date           object
Respondents    object
url            object
dtype: object

In [14]:
df_litreleases_17 = df_litreleases_17[['Release No.','Respondents','Date','url', 'full_text']]
df_litreleases_17.head()

Release No.  \
0  LR-24022     
1  LR-24021     
2  LR-24020     
3  LR-24019     
4  LR-24018     

                                                                                                                   Respondents  \
0  Gaughran, et al. See also: Complaint                                                                                          
1  Anvil Partners, Inc. See also: Contempt Order                                                                                 
2  Robert H. Shapiro, Woodbridge Group of Companies, LLC, et al. See also: SEC Complaint                                         
3  Stephen C. Peters, VisionQuest Wealth Management, LLC, VisionQuest Capital, LLC, and VQ Wealth, LLC See also: SEC Complaint   
4  Matrix Capital Markets, LLC, Nicholas M. Mitsakos and Courtlin L. Holt-Nguyen                                                 

            Date                                                          url  \
0  Dec. 26, 2017  https://www.sec.gov/litigation/litreleases/2017/lr24022.htm   
1  Dec. 26, 2017  https://www.sec.gov/litigation/litreleases/2017/lr24021.htm   
2  Dec. 21, 2017  https://www.sec.gov/litigation/litreleases/2017/lr24020.htm   
3  Dec. 21, 2017  https://www.sec.gov/litigation/litreleases/2017/lr24019.htm   
4  Dec. 20, 2017  https://www.sec.gov/litigation/litreleases/2017/lr24018.htm   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [15]:
# Save to CSV
df_litreleases_17.to_csv("litreleases_2017_scraped.csv", index=False)